### **4.14 - How to test your DAGs**

Un aspecto muy importante a tener en cuenta son las pruebas unitarias (unit testing). Si estás trabajando en una empresa sabes que es crucial hacer pruebas unitarias para estar seguro de que nada se rompe después de que su aplicación ha sido modificada. Es aún más importante cuando puedes trabajar con diferentes equipos. En este video, voy a mostrarte cómo puedes probar tus DAGs para que puedas ponerlos en producción sin tener miedo de romper todo. Vamos a empezar. Hay muchas herramientas de prueba de Python, pero en nuestro caso vamos a centrarnos en Pytest. Pytest es un framework de pruebas que permite escribir códigos de prueba muy simples, así como complejas pruebas funcionales para aplicaciones y librerías. Si te preguntas por qué no elegí unittest, es porque Pytest está diseñado para ser más simple, y más fácil de codificar que unittest y es ampliamente utilizado por la comunidad. Pytest será más que suficiente para probar nuestros DAGs. Por cierto, no dudes en consultar su documentación si quieres aprender más sobre él.

<center><img src="https://i.postimg.cc/YC0RP5dr/a736.png"></center>

Ahora mismo tenemos el testing framework, la pregunta es, ¿qué podemos probar en Airflow? Bueno, para probar nuestros DAGs, en mi opinión, hay cinco categorías de pruebas que se pueden implementar.

<center><img src="https://i.postimg.cc/GmfKnXfk/a737.png"></center>

Empecemos por la primera. DAG Validation Tests (pruebas de validación de DAGs) están pensadas para ser pruebas comunes para todos tus DAGs para comprobar si son válidos. Básicamente, estas pruebas verificarán si hay algún error tipográfico en tus DAGs. Si sus DAGs contienen o no ciclos, lo que significa que tu DAG no podrá terminar. Si los argumentos por defecto están correctamente configurados, por ejemplo, la dirección de correo electrónico para las alertas, etc. El objetivo de estas pruebas es evitar que aparezca un mensaje en la UI de Airflow diciendo que uno de sus DAGs está roto.

<center><img src="https://i.postimg.cc/sXdJvvsx/a738.png"></center>

La siguiente categoría es la de DAG/Pipeline Definition Tests (Pruebas de Definición de DAG/Pipeline). Como puede adivinar por el nombre de la categoría, se utilizarán para comprobar la definición de sus DAGs. El objetivo no es verificar la lógica de tus DAGs sino sólo si las modificaciones que hiciste fueron intencionales o no. En estas pruebas comprobaremos cuántas tareas tiene tu DAG, la naturaleza de las tareas, si las dependencias se establecen con las tareas correctas, etc. A diferencia de la categoría anterior, aquí tendremos un conjunto de pruebas para cada DAG.

<center><img src="https://i.postimg.cc/PxWQZT54/a739.png"></center>

Después de haber comprobado las definiciones del DAG, tenemos las unit tests (pruebas unitarias). Dado que Airflow es sólo un orquestador y no un framework de procesamiento de datos como Spark no debería tener demasiada lógica en sus operators. La parte de procesamiento debe ser externalizada en sus herramientas de big data y Airflow sólo debe llamarlos en el orden correcto. Como mejor práctica, mantenga sus operators pequeños y limpios con una tarea y trate de evitar en lo posible mover o cambiar datos a través de ellos. Por lo tanto, los únicos operators o sensors que debe probar son los que crea por su cuenta. No me malinterpretes, no estoy diciendo que no debas probar el código que procesa tus datos, por supuesto que debes hacerlo, pero no dentro de Airflow. Si estás llamando a Spark desde Airflow, deberías hacer tus pruebas unitarias dentro de Spark y así sucesivamente. En resumen, cuando creas un nuevo operator o un sensor, debes comprobar si funciona como se espera utilizando pruebas unitarias.

<center><img src="https://i.postimg.cc/GpsQLk06/a740.png"></center>

Una vez realizadas las pruebas unitarias, podemos pasar a las integration tests (pruebas de integración). Una prueba de integración es una forma diferente de prueba en la que se comprueban explícitamente las interacciones entre dos o más tareas. Las pruebas de integración verifican que las tareas de tu DAG funcionan juntas. Por ejemplo, se comprueba si una tarea puede obtener los datos necesarios para trabajar, si dos o más tareas pueden intercambiar datos entre ellas, si las conexiones o cualquier recurso externo está disponible desde una tarea determinada, etc. Las pruebas de integración pueden ser complejas y más lentas que el resto de pruebas, ya que tendremos que tratar con herramientas externas como Spark o Postgres. Básicamente, necesitaremos configurar un entorno diferente llamado "test" por ejemplo, donde utilizaremos un subconjunto de los datos de producción para ejecutar las pruebas.

<center><img src="https://i.postimg.cc/J7QQzrg7/a741.png"></center>

Una vez que las pruebas de integración se han ejecutado con éxito, podemos pasar a la última categoría de pruebas que son los end to end pipeline tests (pruebas de pipeline de extremo a extremo). Los end to end pipeline tests son las últimas pruebas que se ejecutan y su objetivo es comprobar toda la lógica de sus DAGs desde la primera tarea hasta la última. Esto permitirá verificar si la salida de un determinado DAG es correcta, si no es demasiado lenta su ejecución, etc. Para que estas pruebas sean útiles, utilizaremos una gran copia de los datos de producción para acercarnos lo más posible a las condiciones de producción. Como en el caso de la categoría anterior, crearemos otro entorno llamado "acceptance" en el que se instalarán las pruebas y todas las herramientas necesarias. Por supuesto, es muy difícil estar seguros de que no tendremos ningún error en producción, pero en este punto, hemos reducido drásticamente las probabilidades de obtener uno.

<center><img src="https://i.postimg.cc/d3Jmjp7K/a742.png"></center>

Así que hemos visto los diferentes tipos de pruebas a definir, vamos a ver los diferentes entornos que vamos a necesitar para configurar. Aquí están. Tenemos el entorno de desarrollo (development environment) en el que utilizaremos pequeños datos falsos para ejecutar las pruebas de validación DAG, las pruebas DAG/Pipeline y las pruebas unitarias. El objetivo de este entorno es verificar que las tareas de nuestros pipelines de datos pueden ser ejecutadas con códigos de salida no nulos. Como los datos que vamos a utilizar como entrada son muy pequeños, las pruebas deberían ejecutarse muy rápidamente. A continuación, tenemos el entorno de "prueba (test)", donde utilizaremos una mayor cantidad de datos para ejecutar las pruebas de integración. Instalaremos todas las herramientas que nuestros DAGs necesitan para ejecutarse, como una base de datos, un framework de procesamiento, etc. Los datos utilizados como entrada no deben ser falsos, ya que queremos ver cómo se comportan las tareas con datos reales. El tercer entorno es el de "aceptación (acceptance)". Utilizaremos una copia completa de los datos en producción para verificar que un determinado DAG funciona como se espera y no se vuelve lento. También será el lugar en el que el propietario del producto de su equipo podrá comprobar que su data pipeline cumple los requisitos definidos antes de pasarlo a producción. Por último, el entorno de producción (production environment) será el entorno de tus usuarios finales (end-users) donde, con suerte, el riesgo de errores se reduce al mínimo.

<center><img src="https://i.postimg.cc/vHHtgRFr/a743.png"></center>

Sólo para mostrarte cómo puedes organizar todo, aquí están las diferentes ramas (branches) de Git con el entorno correspondiente. Por ejemplo, cada vez que tú o un ingeniero de datos de tu equipo quiera modificar un DAG, deberá crear una nueva rama llamada "feature". Una vez que el trabajo está hecho, su código es revisado y fusionado (merged) en la rama de desarrollo (development branch). Entonces, cuando la fusión se produce, la herramienta de CI ejecuta las pruebas que definimos antes. Si todas pasan, el DAG se fusiona automáticamente con la rama "test". A partir de ahí, se ejecutan las pruebas de integración (integration tests). Una vez que se han superado, la herramienta CI fusiona la rama de "test" con la rama de "acceptance". En este punto, el DAG no se desplegará automáticamente en producción. Como mejor práctica, es mejor que el product owner compruebe que todo funciona como se espera y que se cumplen los requisitos estándar. Si está bien, se abre una solicitud de fusión (merge request) en la rama maestra (master branch) para una última comprobación. A continuación, validamos manualmente la solicitud de fusión (merge) y empujamos las modificaciones en producción. Y eso es todo, el proceso está hecho.

<center><img src="https://i.postimg.cc/KjQ7qfL3/a744.png"></center>

Antes de seguir adelante, quería darte los siguientes comandos que creo que puedes encontrar útiles para probar tus DAGs.

<center><img src="https://i.postimg.cc/wTfcDWgS/a745.png"></center>